In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
import psycopg2
import pickle
from sklearn import preprocessing
import credentials

In [17]:
con=psycopg2.connect(dbname= credentials.dbname, host=credentials.host, 
port= credentials.port, user= credentials.user, password= credentials.password)

In [20]:
fd = open(r'/Users/miqui/Documents/Miqui/Preply/Tutor Performance/Projects/Q3/TWA/Project.ipynb', 'r')
twa_query = fd.read()
fd

<_io.TextIOWrapper name='/Users/miqui/Documents/Miqui/Preply/Tutor Performance/Projects/Q3/TWA/Project.ipynb' mode='r' encoding='UTF-8'>

In [26]:
file = open('/Users/miqui/Documents/Miqui/Preply/Tutor Performance/Projects/Q3/TWA/TWA_Query.sql').read()

In [27]:
data = pd.read_sql(file, con)



In [30]:
data.describe()

,tutor_id,student_id,duration,is_back_to_back_lesson,booking_window,lesson_id,tutor_lesson_order,days_since_last_seen,local_lesson_hour,hour_offset_tutor,hour_offset_student,hour_differences_tutor_student,local_booking_hour,num_lessons_per_day,one_lesson_per_day,app_usage,response_time
count,3.423070e+05,3.423070e+05,342307.000000,342307.000000,342307.000000,3.423070e+05,342307.000000,307258.000000,342307.000000,342307.000000,342275.000000,342275.000000,342307.000000,342307.000000,342307.000000,342307.000000,241799.000000
mean,7.747663e+05,2.817090e+06,0.999623,0.051363,82.787133,1.431284e+07,776.119700,2.101807,14.409536,0.654047,-0.170369,3.830852,13.302226,1.607671,0.657173,0.429451,6.852768
std,4.834812e+05,8.770846e+05,0.023252,0.220738,86.234213,2.092263e+06,1575.258528,10.764174,4.952770,4.305341,4.434083,4.070261,6.422135,1.211759,0.474655,0.494999,28.499487
min,1.100000e+01,5.460000e+02,0.250000,0.000000,-362.000000,8.255131e+06,1.000000,0.000000,0.000000,-11.000000,-11.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,3.171000e+05,2.510312e+06,1.000000,0.000000,31.000000,1.248666e+07,24.000000,1.000000,11.000000,-2.000000,-5.000000,1.000000,9.000000,1.000000,0.000000,0.000000,1.350000
50%,7.908640e+05,3.017002e+06,1.000000,0.000000,55.000000,1.410314e+07,160.000000,1.000000,15.000000,1.000000,1.000000,2.000000,14.000000,1.000000,1.000000,0.000000,2.850000
75%,1.174245e+06,3.460227e+06,1.000000,0.000000,103.000000,1.614530e+07,789.000000,2.000000,18.000000,2.000000,2.000000,6.000000,19.000000,2.000000,1.000000,1.000000,5.810000
max,1.805715e+06,3.956579e+06,3.500000,1.000000,5732.000000,1.829060e+07,18291.000000,1275.000000,23.000000,11.000000,12.000000,22.000000,23.000000,24.000000,1.000000,1.000000,1872.020000


In [ ]:
fd = open(r'C:\Users\Lenovo\Documents\to forecast twa.sql', 'r')
twa_query = fd.read()
fd.close()
conn_string = "host="+ creds.PGHOST +" port="+ creds.PGPORT +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
+" password="+ creds.PGPASSWORD
conn=psycopg2.connect(conn_string)
data = pd.read_sql(twa_query, conn)
file = open("twa_20210514.pkl",'rb')
twa_model = pickle.load(file)
file.close()
data['twa_prob'] = twa_model.predict_proba(data[twa_model.get_booster().feature_names])[:,1]
data['is_bad_twa'] = 1*(data['twa_prob']>=0.0875) # Buscar la mínima probabilidad e un tutor que ha hecho TWA.
data = data[['tutor_id', 'twa_prob', 'is_bad_twa']]

#and modelling code:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier,XGBRegressor
from xgboost import plot_tree
#import graphviz
#from sklearn import cross_validation, metrics   #Additional scklearn functions
#from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.model_selection import learning_curve,GridSearchCV,train_test_split
import matplotlib.pylab as plt
import matplotlib.pyplot as plt1
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,roc_auc_score
#import sys, os
data = pd.read_csv('C:/Users/Lenovo/Downloads/twa_file.csv',delimiter=";",decimal=",")
data1 = data.head()
X = data.drop(['is_twa', 'flid'], axis=1)#.head(1000)
Y = data[['is_twa']]#.head(1000)
#X = X.head(3000)
#Y = Y.head(3000)

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
model = XGBClassifier(max_depth=5,
                           min_child_weight=9,
                           learning_rate=0.05,
                           n_estimators=80,
                           objective='binary:logistic',
                           eval_metric = 'auc',
                           gamma=0,
                           max_delta_step=0,
                           subsample=0.8,
                           colsample_bytree=0.86,
                           colsample_bylevel=0.67,
                           reg_alpha=0,
                           reg_lambda=2.8,
                           scale_pos_weight=1,
                           seed=1)
eval_set  = [( X_train, y_train), ( X_test, y_test)]
model.fit(X_train, y_train,eval_set=eval_set,early_stopping_rounds=10)
# import pickle
# file_name = "twa_20210514.pkl"
# # save
# pickle.dump(model, open(file_name, "wb"))
y_pred = model.predict_proba(X_test)[:,1]